In [1]:
import tensorflow as tf
import sys
import os
import re
import pydub as pdb

/home/aeatda/anaconda3/envs/proj3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/aeatda/anaconda3/envs/proj3/lib/python3.6/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
def get_array(filetype='wav'):
    namelist, dflist, ylist = [], [], []
    for category in ['on', 'off']:
        cat_path = '../_data/{}/'.format(category)
        names = [name for name in os.listdir(cat_path)
                     if re.search(r'\.{}$'.format(filetype), name)]
        for name in names:
            f1 = pdb.AudioSegment.from_wav(cat_path+name).get_array_of_samples().tolist()
            if len(f1) == 16000:
                namelist.append(name[:-4])
                dflist.append(f1)
                ylist.append(category=='on')
    return namelist, dflist, ylist

In [3]:
def mu_encode(x, mu=255, bitdepth=16):
    x = np.divide(x, np.power(2, bitdepth-1))
    x_scale = np.sign(x) * np.log(1 + mu*np.abs(x)) / np.log(1 + mu)
    return np.round((x_scale + 1) / 2 * mu)

In [4]:
%%time
namelist, dflist, ylist = get_array()

df_mu = pd.DataFrame(mu_encode(np.array(dflist)), columns=range(1, 16001))
df_mu.insert(0, 0, namelist)
df_mu.insert(16001, 16001, ylist)

CPU times: user 7.42 s, sys: 2.37 s, total: 9.79 s
Wall time: 11.9 s


In [5]:
df_mu.shape

(4248, 16002)

In [5]:
df_s = df_mu.sample(frac=1).reset_index()

In [6]:
del df_s[0]

In [7]:
df_train = df_s.iloc[:4000, :].copy()
df_test = df_s.iloc[4000:4240, :].copy()

df_train_val = df_train.values.astype(np.int64)
df_test_val = df_test.values.astype(np.int64)

In [8]:
data_path0 = '../_data/tfrecords/sample_{}_16000_{}.tfrecord'

def write_tfrecord(arr, topic='onoff', name='train'):
    global data_path0
    data_path = data_path0.format(topic, name)
    print(data_path)
    num_samples = arr.shape[1] - 2

    with tf.python_io.TFRecordWriter(data_path) as writer:
        for row in arr:
            idx, samples, y\
            = row[0], row[1:-1], row[-1]

            example = tf.train.Example()
            example.features.feature["idx"].int64_list.value.append(idx)
            example.features.feature["samples"].float_list.value.extend(samples)
            example.features.feature["y"].float_list.value.append(y)

            writer.write(example.SerializeToString())
    return

In [9]:
%%time
write_tfrecord(df_train_val, name='train')

../_data/tfrecords/sample_onoff_16000_train.tfrecord
CPU times: user 12.4 s, sys: 126 ms, total: 12.5 s
Wall time: 14.3 s


In [10]:
%%time
write_tfrecord(df_test_val, name='test')

../_data/tfrecords/sample_onoff_16000_test.tfrecord
CPU times: user 754 ms, sys: 12.7 ms, total: 767 ms
Wall time: 854 ms


In [11]:
num_samples = 16000

In [12]:
def parser(serialized_example):
    features = {
        'idx' : tf.FixedLenFeature([1], tf.int64),
        'samples': tf.FixedLenFeature([num_samples], tf.float32),
        'y' : tf.FixedLenFeature([1], tf.float32)
    }
    parsed_feature = tf.parse_single_example(serialized_example, features)
    samples = parsed_feature['samples']
    y = parsed_feature['y']
    idx = parsed_feature['idx']
    return idx, samples, y

def get_tfrecord(topic='onoff', name='train', sample_size=16000, batch_size=1, buffer_size=5000,
                 repeat=1, seed=None):
    dataset = tf.data.TFRecordDataset(data_path0.format(topic, name)).map(parser)
    dataset = dataset.batch(batch_size)
    dataset = dataset.shuffle(buffer_size, seed=seed)
    dataset = dataset.repeat(repeat)
    return dataset

In [12]:
def get_tfrecord(topic='onoff', name='train', sample_size=16000, batch_size=1, buffer_size=5000,
                 repeat=1, seed=None, data_path=None):
    def parser(serialized_example):
        features = {
            'idx' : tf.FixedLenFeature([1], tf.int64),
            'samples': tf.FixedLenFeature([sample_size], tf.float32),
            'y' : tf.FixedLenFeature([1], tf.float32)
        }
        parsed_feature = tf.parse_single_example(serialized_example, features)
        samples = parsed_feature['samples']
        y = parsed_feature['y']
        idx = parsed_feature['idx']
        return idx, samples, y
    
    dataset = tf.data.TFRecordDataset(data_path.format(topic, sample_size, name)).map(parser)
    dataset = dataset.batch(batch_size)
    dataset = dataset.shuffle(buffer_size, seed=seed)
    dataset = dataset.repeat(repeat)
    return dataset

In [13]:
data_train = get_tfrecord(batch_size=1, seed=0)
itr = data_train.make_one_shot_iterator()
idx, sample_batch, y = itr.get_next()
sample_batch = tf.reshape(sample_batch, [-1, sample_batch.shape[1], 1])

In [14]:
sess = tf.Session()
idces, batches, ys = [], [], []

In [15]:
i, bat, yy = sess.run([idx, sample_batch, y])
idces.extend(i)
batches.extend(bat)
ys.extend(yy)

In [16]:
ys

[array([ 0.], dtype=float32)]